![alt text](./Cerny_logo_1.jpg)

# The effect of ambulance acceleration on mechanical ventilation during neonatal transport

#### Author: Dr Gusztav Belteki

### 1. Import the required libraries and set options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import pickle

from scipy import stats
from pandas import Series, DataFrame
from datetime import datetime, timedelta
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'
#matplotlib.rcParams['date.autoformatter.second'] = '%H:%M:%S.%f'

pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 100)

In [ ]:
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("seaborn version: {}".format(sns.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))
print("IPython version: {}".format(IPython.__version__))

### 2. List and set the working directory and the directory to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'accelerometer_ventilated'

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Directory containing clinical and blood gas data
CWD = os.path.join('/Users', 'guszti', 'ventilation_fabian')

DIR_WRITE = os.path.join(CWD, 'Analyses', TOPIC)
DATA_DUMP = os.path.join('/Volumes', DRIVE, 'data_dump', 'fabian')

In [ ]:
DIR_WRITE, DATA_DUMP

### 3. Import ventilator, accelerometer and clinical data from pickle archives

In [ ]:
%%time 
with open('%s/%s.pickle' % (DATA_DUMP, 'clin_df_1_1100'), 'rb') as handle:
    clin_df = pickle.load(handle)

In [ ]:
%%time
with open('%s/%s.pickle' % (DATA_DUMP, 'combined_ventilated_1min'), 'rb') as handle:
    combined_ventilated_1min = pickle.load(handle)  

In [ ]:
%%time
with open('%s/%s.pickle' % (DATA_DUMP, 'ventilator_mode'), 'rb') as handle:
    ventilator_mode = pickle.load(handle)

In [ ]:
%%time

with open('%s/%s.pickle' % (DATA_DUMP, 'accelero_ventilated_2'), 'rb') as handle:
    accelero_ventilated = pickle.load(handle)

with open('%s/%s.pickle' % (DATA_DUMP, 'accelero_ventilated_low_pass'), 'rb') as handle:
    accelero_ventilated_low_pass = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'accelero_ventilated_high_pass'), 'rb') as handle:
    accelero_ventilated_high_pass = pickle.load(handle)

### 4. Remove those recordings whose postnatal age is over term + 6 weeks

In [ ]:
patients = [key[1] for key in combined_ventilated_1min.keys()]
len(patients)

In [ ]:
clin_df = clin_df.loc[patients]

In [ ]:
len(clin_df)

In [ ]:
clin_df[clin_df['Corrected gestational Age (weeks)'] > 46]

In [ ]:
clin_df = clin_df[clin_df['Corrected gestational Age (weeks)'] <= 46]
len(clin_df)

In [ ]:
patients = sorted(clin_df.index)
len(patients)

### 5. Statistics on clinical details of the final set of patients

The _postnatal age_ shown in the table is the age at the end of the transfer

In [ ]:
clin_df.info()

In [ ]:
clin_df_stats = round(clin_df.describe(percentiles = [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 2)
clin_df_stats

#### Boxplots on selected clinical parameters 

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['Gestational age']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot(clin_df['Gestational Age (weeks)'] ,
    widths = 0.3, whis = [0, 100], showfliers = True, showmeans = True, medianprops=medianprops, 
    meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, 
    flierprops = flierprops)

ax.set_ylabel('weeks', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'accelero_gest_age', filetype),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['Birth weight']

medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot(clin_df['Birth Weight'], widths = 0.3, whis = [0, 100], showfliers = True,
             showmeans = True, meanprops = meanpointprops, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylabel('grams', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'accelero_birth_weight', filetype),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
below_29 = sorted(clin_df[clin_df['Corrected gestational Age (weeks)'] < 29].index)
len(below_29)

In [ ]:
between_29_32 = sorted(clin_df[(clin_df['Corrected gestational Age (weeks)'] >= 29) & 
                       (clin_df['Corrected gestational Age (weeks)'] < 33)].index)
len(between_29_32)

In [ ]:
between_33_36 = sorted(clin_df[(clin_df['Corrected gestational Age (weeks)'] >= 33) & 
                       (clin_df['Corrected gestational Age (weeks)'] < 37)].index)
len(between_33_36)

In [ ]:
between_37_40 = sorted(clin_df[(clin_df['Corrected gestational Age (weeks)'] >= 37) & 
                       (clin_df['Corrected gestational Age (weeks)'] < 41)].index)
len(between_37_40)

In [ ]:
over_40 = sorted(clin_df[clin_df['Corrected gestational Age (weeks)'] >= 41].index)
len(over_40)

#### Clinical problems

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'clinical_data_accelerometer_final.xlsx'))
clin_df.to_excel(writer, 'cases')
clin_df_stats.to_excel(writer, 'stats')
writer.save()

### 6. Remove the samples with  >46 week corrected gestational age from the ventilator and accelerometer data as well

In [ ]:
combined_ventilated_1min = {key: value for key, value in combined_ventilated_1min.items() 
                               if key[1] in patients}

accelero_ventilated = {key: value for key, value in accelero_ventilated.items() 
                               if key[1] in patients}

accelero_ventilated_low_pass = {key: value for key, value in accelero_ventilated_low_pass.items() 
                               if key[1] in patients}

accelero_ventilated_high_pass = {key: value for key, value in accelero_ventilated_high_pass.items() 
                               if key[1] in patients}

ventilator_mode = {key: value for key, value in ventilator_mode.items() if key[1] in patients}

In [ ]:
len(combined_ventilated_1min),  len(ventilator_mode)

In [ ]:
len(accelero_ventilated), len(accelero_ventilated_low_pass), len(accelero_ventilated_high_pass),

### 7. Calculate recording durations of the final dataset

In [ ]:
recording_durations_final = {}

for recording in combined_ventilated_1min:
    recording_durations_final[recording] = len(combined_ventilated_1min[recording])
    
recording_durations_final = DataFrame(Series(recording_durations_final), columns = ['duration (minutes)'])
recording_durations_final.head()

In [ ]:
recording_durations_final.describe()

In [ ]:
recording_durations_final.sum(), recording_durations_final.sum() / 60, recording_durations_final.sum() / 3600, 

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'recording_durations_final.xlsx'))
recording_durations_final.to_excel(writer, 'final')
recording_durations_final.describe().to_excel(writer, 'stats')
writer.save()

### 8. Analyse ventilator modes

Only consider those recordings where the ventilation mode did not change during the recording

In [ ]:
simv_only = {key[1] for key, value in ventilator_mode.items()
           if 'SIMV' in value['Ventilator_mode'].unique() and len(value['Ventilator_mode'].unique()) == 1}
len(simv_only)

In [ ]:
sippv_only = {key[1] for key, value in ventilator_mode.items()
           if 'SIPPV' in value['Ventilator_mode'].unique() and len(value['Ventilator_mode'].unique()) == 1}
len(sippv_only)

In [ ]:
simvpsv_only = {key[1] for key, value in ventilator_mode.items()
           if 'SIMVPSV' in value['Ventilator_mode'].unique() and len(value['Ventilator_mode'].unique()) == 1}
len(simvpsv_only)

How many recording has multiple modes

In [ ]:
multiple_mode = {key[1] for key, value in ventilator_mode.items()
           if len(value['Ventilator_mode'].unique()) > 1}
len(multiple_mode)

How many times was VG on for part or all of the recording

In [ ]:
has_VG = set()

for recording in combined_ventilated_1min:
    if 'VG_set_kg_mean' in combined_ventilated_1min[recording].columns:
        has_VG.add(recording)
        # print(recording)

len(has_VG)

In [ ]:
has_noVG = set()

for recording in combined_ventilated_1min:
    if 'VG_set_kg_mean' not in combined_ventilated_1min[recording].columns:
        has_noVG.add(recording)
    
    elif sum(combined_ventilated_1min[recording]['VG_set_kg_mean'].isnull()):
        has_noVG.add(recording)
    
len(has_noVG)

### 9. Generate explanatory graphs for acceleration vectors

In [ ]:
filetype = 'jpg'
dpi  = 200

fig = plt.figure(figsize = [6,6])
ax = fig.add_subplot(111, projection='3d')
ax.quiver(0, 0, 0, 3.5, 2.8, 3.5, linewidth = 2, color = 'red', linestyle = '-', arrow_length_ratio = 0.1)
ax.quiver(0, 0, 0, 3.5, 0, 0, linewidth = 2, color = 'black', arrow_length_ratio = 0.1)
ax.quiver(0, 0, 0, 0, 2.8, 0, linewidth = 2, color = 'black', arrow_length_ratio = 0.1)
ax.quiver(0, 0, 0, 0, 0, 3.5, linewidth = 2, color = 'black', arrow_length_ratio = 0.1)

ax.quiver(0, 2.8, 0, 0, 0, 3.5, linewidth = 2, linestyle = 'dotted', color = 'black', arrow_length_ratio = 0.1,)
ax.quiver(0, 2.8, 3.5, 3.5, 0, 0, linewidth = 2, linestyle = 'dotted', color = 'black', arrow_length_ratio = 0.1,)

ax.set_xlim([0, 5]); ax.set_ylim([0,4]); ax.set_zlim([0, 4])
ax.set_xlabel('X (m/sec$^2$)'); ax.set_ylabel('Y (m/sec$^2$)'); ax.set_zlabel('Z (m/sec$^2$)')
ax.view_init(elev=20., azim = 200)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'accel_explanation', filetype), 
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

### 10. Generate and export histograms of accelerations using the original accelerometer data (~100 Hz sampling)

### A. Acceleration vectors

In [ ]:
def acceleration(rec, direction, bins = 50, xlim = [-3, 3], ylim = [0, 100000], logyscale = False,
               color = 'blue', show = True, save = False, dpi = 200, filetype = 'jpg'):
    
    fig, ax = plt.subplots(figsize = [4, 4])
    
    accelero_ventilated[rec][direction].hist(ax = ax, bins=bins, color = color)
    ax.set_xlabel('m/sec$^2$', size = 12)
    ax.set_ylabel('count', size = 12)
    ax.set_title(direction, size = 14)
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    
    if logyscale:
        ax.set_yscale('log')
    
    name = rec[1] + '_log' if logyscale else rec[1]
    
    if save:       
        fig.savefig('%s/%s_%s_%s.%s' % (DIR_WRITE, 'hist', name, direction, filetype), dpi = dpi, 
           format = filetype, bbox_inches='tight', pad_inches=0.1)
        
    if not show:
        plt.close()

In [ ]:
acceleration(('default__393.txt', 'AL000628'), 'X', bins = 20, logyscale = True, xlim = [-8,8],
             ylim = [1, 1000000], save = True)

In [ ]:
acceleration(('default__393.txt', 'AL000628'), 'Y', bins = 50, logyscale = True, xlim = [-9,9],
             ylim = [1, 1000000], save = True)

In [ ]:
acceleration(('default__393.txt', 'AL000628'), 'Z', bins = 30, logyscale = True, xlim = [-10,12],
             ylim = [1, 1000000], save = True)

#### Absolute value of acceleration vectors

In [ ]:
def acceleration_abs(rec, direction, bins = 50, xlim = [0, 8], ylim = [0, 100000], logyscale = False,
               color = 'blue', show = True, save = False, dpi = 200, filetype = 'jpg'):
    
    fig, ax = plt.subplots(figsize = [4, 4])
    
    accelero_ventilated[rec]['%s_abs' % direction].hist(ax = ax, bins=bins, color = color)
    ax.set_xlabel('m/sec$^2$', size = 12)
    ax.set_ylabel('count', size = 12)
    ax.set_title(direction, size = 14)
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    
    if logyscale:
        ax.set_yscale('log')
    
    name = rec[1] + '_log' if logyscale else rec[1]
    
    if save:       
        fig.savefig('%s/%s_%s_%s.%s' % (DIR_WRITE, 'hist_abs', name, direction, filetype), dpi = dpi, 
        format = filetype, bbox_inches='tight', pad_inches=0.1)
        
    if not show:
        plt.close()

In [ ]:
acceleration_abs(('default__393.txt', 'AL000628'), 'X', bins = 20, logyscale = True, xlim = [0,7],
             ylim = [1, 1000000], save = True)

In [ ]:
acceleration_abs(('default__393.txt', 'AL000628'), 'Y', bins = 20, logyscale = True, xlim = [0,10],
             ylim = [1, 1000000], save = True)

In [ ]:
acceleration_abs(('default__393.txt', 'AL000628'), 'Z', bins = 20, logyscale = True, xlim = [0,12],
             ylim = [1, 1000000], save = True)

#### X` (front-back), `Y` (left-right) and `Z` (up-down) acceleration vectors together

In [ ]:
def acceleration_XYZ(rec, bins = 50, xlim = [-3, 3], logyscale = False,
               color = 'blue', show = True, save = False, dpi = 200, filetype = 'jpg'):
    
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize = [12, 4], sharex=False, sharey=True)
    
    for par, ax in zip(['X', 'Y', 'Z'], axes.ravel()):
        
        data = accelero_ventilated[rec][par]
        # Remove outlier data outside the X axis scale
        data = data[(data > xlim[0]) & (data < xlim[1])]
        data.hist(ax = ax, bins=bins, color = color)
        ax.set_xlabel('m/sec$^2$', size = 14)
        ax.set_ylabel('count', size = 14)
        ax.set_title(par, size = 16)
        ax.set_xlim(*xlim)
    
        if logyscale:
            ax.set_yscale('log')
    
    name = rec[1] + '_log' if logyscale else rec[1]
    
    if save:
        fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'hist_XYZ', name, filetype), dpi = dpi, 
        format = filetype, bbox_inches='tight', pad_inches=0.1)
        
    if not show:
        plt.close()

In [ ]:
# Example
acceleration_XYZ(('default__393.txt', 'AL000628'), bins = 50, save = True)

In [ ]:
# Example
acceleration_XYZ(('default__393.txt', 'AL000628'), bins = 50, xlim = (-10, 10), logyscale=True, save = True)

In [ ]:
def accelerometer_XYZ_all(recs, bins = 50, xlim = [-3, 3], logyscale = False, color = 'blue', 
    name = 'hist_XYZ_all', save = True, show = False, dpi = 200, filetype = 'jpg'):
    '''
    Generates a multiplot graph with the acceleration of all recordings with each recording in a separate
    row. This results in a very large vertical side length of the image
    '''
    fig, axes = plt.subplots(nrows=len(recs), ncols=3, figsize = [12, len(recs) * 4], sharex=False, sharey=False)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=0.5)
    
    ax_ravelled = axes.ravel()
    i = 0
    for rec in recs:
        for par in ['X', 'Y', 'Z']:
            ax = ax_ravelled[i]
            data = accelero_ventilated[rec][par]
            # Remove outlier data outside the X axis scale
            data = data[(data > xlim[0]) & (data < xlim[1])]
            data.hist(ax = ax , bins=bins, color = color)
            ax.set_xlabel('m/sec$^2$', size = 10)
            ax.set_ylabel('count', size = 10)
            ax.set_title('%s - %s' % (rec[1], par), size = 10)
            ax.set_xlim([*xlim])
            
            if logyscale:
                ax.set_yscale('log')
            
            i+=1
            if i % 10 == 0:
                print(datetime.now(), 'Created %s graphs' %i)
            
    name = name + '_log' if logyscale else name
    
    if save:
        fig.savefig('%s/%s.%s' % (DIR_WRITE, name, filetype), dpi = dpi, 
        format = filetype, bbox_inches='tight', pad_inches=0.1)
        
    if not show:
        plt.close()

In [ ]:
# Example plot
accelerometer_XYZ_all([('default__369.txt', 'AL000611'), ('default__393.txt', 'AL000628')], 
                      show = True, save= False)

In [ ]:
# Example plot
accelerometer_XYZ_all([('default__369.txt', 'AL000611'), ('default__393.txt', 'AL000628')], 
                      xlim = (-10, 10), logyscale=True, show = True, save= False)

In [ ]:
def accelerometer_XYZ_all_2(recs, bins = 50, xlim = [-3, 3], logyscale = False, color = 'blue',
    name = 'hist_XYZ_all', save = True, show = False, dpi = 200, filetype = 'jpg'):
    '''
    Generates a multiplot graph with the acceleration of all recordings with each recording in a separate
    row. This results in a very large vertical side length of the image
    '''
    fig, axes = plt.subplots(nrows=len(recs) // 4+1, ncols=12, figsize = [60, 60], sharex=False, sharey=False)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.6, hspace=0.6)
    
    ax_ravelled = axes.ravel()
    i = 0
    for rec in recs:
        for par in ['X', 'Y', 'Z']:
            ax = ax_ravelled[i]
            data = accelero_ventilated[rec][par]
            # Remove outlier data outside the X axis scale
            data = data[(data > xlim[0]) & (data < xlim[1])]
            data.hist(ax = ax , bins=bins, color = color)
            ax.set_xlabel('m/sec$^2$', size = 10)
            ax.set_ylabel('count', size = 10)
            ax.set_title('%s - %s' % (rec[1], par), size = 10)
            ax.set_xlim([*xlim])
            
            if logyscale:
                ax.set_yscale('log')
            
            i+=1
            if i % 10 == 0:
                print(datetime.now(), 'Created %s graphs' %i)
            
    name = name + '_log' if logyscale else name
    
    if save:
        fig.savefig('%s/%s.%s' % (DIR_WRITE, name, filetype), dpi = dpi, 
        format = filetype, bbox_inches='tight', pad_inches=0.1)
        
    if not show:
        plt.close()

### B. Length (Euclidean (L2) norm) of the acceleration vectors

In [ ]:
def accelerometer_euclidean_norm(rec, bins = 50, xlim = [0, 4], ylim = [0, 40000],
        logyscale = False, color = 'blue', show = True, save = False, dpi = 200, filetype = 'jpg'):
    
    fig, ax = plt.subplots(figsize = [4, 4])
    
    data = accelero_ventilated[rec]['length']
    # Remove outlier data outside the X axis scale
    data = data[(data > xlim[0]) & (data < xlim[1])]
    data.hist(ax = ax, bins=bins, color = color)
    ax.set_xlabel('m/sec$^2$', size = 12)
    ax.set_ylabel('count', size = 12)
    ax.set_title('Euclidean length', size = 14)
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    
    if logyscale:
        ax.set_yscale('log')
    
    name = rec[1] + '_log' if logyscale else rec[1]
    
    if save:       
        fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'hist_euclidean', name, filetype), dpi = dpi, 
            format = filetype, bbox_inches='tight', pad_inches=0.1)
        
    if not show:
        plt.close()

In [ ]:
# Example
accelerometer_euclidean_norm(('default__393.txt', 'AL000628'), bins = 50, save = True)

In [ ]:
accelerometer_euclidean_norm(('default__393.txt', 'AL000628'), bins = 30,
                             logyscale=True, xlim = (0,10), ylim = (1, 1000000), save = True)

### 11. Analyse distributions of acceleration vector components in the whole recordings

#### Median of absolute values of acceleration vectors and the euclidean length in each recording overall

In [ ]:
to_keep = ['X_abs', 'Y_abs', 'Z_abs', 'length']

accelero_stats_median = {}
for recording in sorted(accelero_ventilated.keys()):
    accelero_stats_median[recording] = accelero_ventilated[recording][to_keep].median()
    
accelero_stats_median = DataFrame(accelero_stats_median).T

accelero_stats_median.head()

In [ ]:
accelero_stats_median_describe = \
    accelero_stats_median.describe(percentiles = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.95, 0.99])
accelero_stats_median_describe

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'median_acceleration_recordings.xlsx'))
accelero_stats_median.to_excel(writer, 'median_accels')
accelero_stats_median_describe.to_excel(writer, 'median_accel_stats')
writer.save()

#### Distribution of the median absolute acceleration of each recording
Medians are not normally distributed as it can be seen on the graphs; use non-parametric statistics: group medians and IQRs

In [ ]:
par = 'X_abs'
data = accelero_stats_median[par]

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 25, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = ax.get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('acceleration (m/sec$^2$)', size = 12)
ax.set_ylabel('count', size = 12)
ax.set_xlim(0, data.mean()*2)
plt.title(par, fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'accel_%s_median_hist_with_pdf' % par, 'jpg' ),
    dpi = 200, format = 'jpg', bbox_inches='tight', pad_inches=0.1,);

In [ ]:
par = 'Y_abs'
data = accelero_stats_median[par]

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 25, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = ax.get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('acceleration (m/sec$^2$)', size = 12)
ax.set_ylabel('count', size = 12)
ax.set_xlim(0, data.mean()*2)
plt.title(par, fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'accel_%s_median_hist_with_pdf' % par, 'jpg' ),
    dpi = 200, format = 'jpg', bbox_inches='tight', pad_inches=0.1,);

In [ ]:
par = 'Z_abs'
data = accelero_stats_median['Z_abs']

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 25, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = ax.get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('acceleration (m/sec$^2$)', size = 12)
ax.set_ylabel('count', size = 12)
ax.set_xlim(0, data.mean()*2)
plt.title(par, fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'accel_Z_abs_median_hist_with_pdf', 'jpg'),
    dpi = 200, format = 'jpg', bbox_inches='tight', pad_inches=0.1,);

In [ ]:
par = 'Euclidean_abs'
data = accelero_stats_median['length']

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 25, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = ax.get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('acceleration (m/sec$^2$)', size = 12)
ax.set_ylabel('count', size = 12)
ax.set_xlim(0, data.mean()*2)
plt.title('Euclidean_abs', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'accel_euclidean_median_hist_with_pdf', 'jpg'),
    dpi = 200, format = 'jpg', bbox_inches='tight', pad_inches=0.1,);

In [ ]:
stats.mannwhitneyu(accelero_stats_median['X_abs'], accelero_stats_median['Y_abs'])

In [ ]:
stats.mannwhitneyu(accelero_stats_median['X_abs'], accelero_stats_median['Z_abs'])

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['X', 'Y', 'Z']

medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([accelero_stats_median['X_abs'], accelero_stats_median['Y_abs'], accelero_stats_median['Z_abs']], 
    widths = 0.3, whis = [5, 95], showfliers = False, showmeans = True, meanprops = meanpointprops, 
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, 
            flierprops = flierprops)

ax.hlines([1.1, 1.3], [1, 1], [2, 3])
ax.vlines([1, 2, 1, 3,], [1.1, 1.1, 1.3, 1.3], [1.05, 1.05, 1.25, 1.25])
ax.text(1.2, 1.15, 'p<0.001'); ax.text(1.55, 1.35, 'p<0.001')

ax.set_xticklabels(xticklabels)
ax.set_ylabel('Median acceleration of recordings \n (m/sec$^2$)', size = 12)
ax.set_ylim(0, 1.5)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'X_Y_Z_median_boxplot', 'jpg'),
    dpi = 200, format = 'jpg', bbox_inches='tight', pad_inches=0.1,);

### 12. Combine individual DataFrames into one

In [ ]:
# combine DataFrames into one
combined_ventilated_1min_all = pd.concat(combined_ventilated_1min, sort = True)
len(combined_ventilated_1min_all)

In [ ]:
# Remove unnecessary columns

columns_to_keep = ['FiO2_set_mean', 'FiO2_set_median', 'FiO2_set_sd', 'Leak_mean', 'Leak_median', 'Leak_sd', 
                   
                   'MAP_mean', 'MAP_sd', 'MV_kg_mean', 'MV_kg_sd', 'MVresp_mean', 'MVresp_sd', 
                   'PIP_mean', 'PIP_sd', 'PIP_set_mean', 'PIP_set_sd',
                   'RR_mean', 'RR_sd', 'RR_set_mean', 'RR_set_sd', 'VG_set_kg_mean', 'VG_set_kg_sd',
                   'VTemand_kg_mean', 'VTemand_kg_sd',
                   'VTemand_resp_kg_mean', 'VTemand_resp_kg_sd', 'VTespon_pat_kg_mean', 'VTespon_pat_kg_sd',
                   'VTimand_kg_mean', 'VTimand_kg_sd', 
                   
                   'Pdiff_VG_abs_mean', 'Pdiff_VG_abs_median', 'Pdiff_VG_abs_sd', 
                   'Pdiff_VG_mean', 'Pdiff_VG_median', 'Pdiff_VG_sd', 
                   'Pdiff_noVG_abs_mean', 'Pdiff_noVG_abs_median', 'Pdiff_noVG_abs_sd', 
                   'Pdiff_noVG_mean', 'Pdiff_noVG_median', 'Pdiff_noVG_sd',
                   
                   'RRdiff_abs_mean', 'RRdiff_abs_median', 'RRdiff_abs_sd', 
                   'RRdiff_mean', 'RRdiff_median', 'RRdiff_sd',
                   
                   'VTdiff_abs_mean', 'VTdiff_abs_median', 'VTdiff_abs_sd', 
                   'VTdiff_mean', 'VTdiff_median', 'VTdiff_sd',
                   
                   'X_pos_median', 'X_neg_median', 'Y_pos_median', 'Y_neg_median',
                   'X_abs_median_low_pass', 'X_abs_median_high_pass', 'Y_abs_median_low_pass', 
                   'Y_abs_median_high_pass', 'Z_abs_median_low_pass', 'Z_abs_median_high_pass', 
                   'length_median_low_pass', 'length_median_high_pass']

combined_ventilated_1min_all = combined_ventilated_1min_all[columns_to_keep]

In [ ]:
combined_ventilated_1min_all.index.rename(['accel_rec', 'patient', 'datetime'], inplace = True)
combined_ventilated_1min_all.reset_index(inplace = True)
combined_ventilated_1min_all.shape

### 13. Analyse the correlation vibration  in X (front-back) or Y (side-to-side) direction with IQR in Z (vertical) direction

- vibration: high-pass filter
- sustained acceleration: low pass filter

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))
dpi = 300
filetype = 'jpg'

a = combined_ventilated_1min_all.dropna(subset = ['Z_abs_median_high_pass','X_abs_median_high_pass'])
corr = stats.spearmanr(a['Z_abs_median_high_pass'], a['X_abs_median_high_pass'])

ax.scatter(combined_ventilated_1min_all['Z_abs_median_high_pass'], 
           combined_ventilated_1min_all['X_abs_median_high_pass'], 
           marker = 'o', facecolor = 'blue', edgecolor = 'blue', alpha = 0.1)
ax.text(0.1, 0.7, 'r: %.2f\np<0.0001' % corr[0], fontsize = 14)
ax.grid(True)
ax.set_xlim(0, combined_ventilated_1min_all['Z_abs_median_high_pass'].max() + 0.05)
ax.set_ylim(-0.02, combined_ventilated_1min_all['X_abs_median_high_pass'].max() + 0.05)
ax.set_xlabel('vertical vibration (m/sec$^2$)', size = 14)
ax.set_ylabel('front-back vibration (m/sec$^2$)', size = 14)
ax.yaxis.set_major_formatter(ScalarFormatter())
ax.tick_params(axis='both', which='major', labelsize=14)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Z_abs_high_pass_X_abs_high_pass', filetype),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))
dpi = 300
filetype = 'jpg'

a = combined_ventilated_1min_all.dropna(subset = ['Z_abs_median_high_pass','Y_abs_median_high_pass'])
corr = stats.spearmanr(a['Z_abs_median_high_pass'], a['Y_abs_median_high_pass'])

ax.scatter(combined_ventilated_1min_all['Z_abs_median_high_pass'], 
           combined_ventilated_1min_all['Y_abs_median_high_pass'], 
           marker = 'o', facecolor = 'blue', edgecolor = 'blue', alpha = 0.1)
ax.text(0.1, 0.6, 'r: %.2f\np<0.0001' % corr[0], fontsize = 14)
ax.grid(True)
ax.set_xlim(0, combined_ventilated_1min_all['Z_abs_median_high_pass'].max() + 0.05)
ax.set_ylim(-0.02, combined_ventilated_1min_all['Y_abs_median_high_pass'].max() + 0.05)
ax.set_xlabel('vertical vibration (m/sec$^2$)', size = 14)
ax.set_ylabel('side-to-side vibration (m/sec$^2$)', size = 14)
ax.yaxis.set_major_formatter(ScalarFormatter())
ax.tick_params(axis='both', which='major', labelsize=14)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Z_abs_high_pass_Y_abs_high_pass', filetype),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

### 14. Visualize the distribution of the median vibration and sustained acceleration along the three axes of the acceleration vector

1 minute medians are compared

- vibration: hig pass filter
- sustained acceleration: low pass filter

In [ ]:
dpi = 300
filetype = 'jpg'

a = combined_ventilated_1min_all.dropna(subset = ['X_abs_median_low_pass','X_abs_median_high_pass'])
#corr = stats.spearmanr(a['X_abs_median_low_pass'], a['X_abs_median_high_pass'])

plot = sns.jointplot(x='X_abs_median_low_pass', y='X_abs_median_high_pass', 
    data = combined_ventilated_1min_all, kind='scatter', color='blue', alpha = 0.1, 
    height=6, ratio=5, space=0.2, dropna=True, xlim=[0, 2], ylim=[0,0.8])

plot.ax_joint.hlines(0.15, 0, 2, color = 'black', linestyle = 'dashed')
#plot.ax_joint.text(0.1, 0.7, 'r: %.2f\np<0.0001' % corr[0], fontsize = 14)
plot.ax_joint.grid('True')
plot.ax_joint.set_xlabel('front-back sustained acceleration (m/sec$^2$)', size = 14)
plot.ax_joint.set_ylabel('front-back vibration (m/sec$^2$)', size = 14)
plot.ax_joint.tick_params(axis='both', which='major', labelsize=14)

plt.savefig('%s/%s.%s' % (DIR_WRITE, 'X_abs_acceleration_X_abs_vibration_jointplot', filetype),
    dpi = dpi,  format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
dpi = 300
filetype = 'jpg'

a = combined_ventilated_1min_all.dropna(subset = ['Y_abs_median_low_pass','Y_abs_median_high_pass'])
#corr = stats.spearmanr(a['Y_abs_median_low_pass'], a['Y_abs_median_high_pass'])

plot = sns.jointplot(x='Y_abs_median_low_pass', y='Y_abs_median_high_pass', 
    data = combined_ventilated_1min_all, kind='scatter', color='blue', alpha = 0.1, 
    height=6, ratio=5, space=0.2, dropna=True, xlim=[0, 1.2], ylim=[0,0.8])

plot.ax_joint.hlines(0.07, 0, 2, color = 'black', linestyle = 'dashed')
#plot.ax_joint.text(0.1, 0.7, 'r: %.2f\np<0.0001' % corr[0], fontsize = 14)
plot.ax_joint.grid('True')
plot.ax_joint.set_xlabel('side-to-side sustained acceleration (m/sec$^2$)', size = 14)
plot.ax_joint.set_ylabel('side-to-side vibration (m/sec$^2$)', size = 14)
plot.ax_joint.tick_params(axis='both', which='major', labelsize=14)

plt.savefig('%s/%s.%s' % (DIR_WRITE, 'Y_abs_acceleration_Y_abs_vibration_jointplot', filetype),
    dpi = dpi,  format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
dpi = 300
filetype = 'jpg'

a = combined_ventilated_1min_all.dropna(subset = ['Z_abs_median_low_pass','Z_abs_median_high_pass'])
#corr = stats.spearmanr(a['Z_abs_median_low_pass'], a['Z_abs_median_high_pass'])

plot = sns.jointplot(x='Z_abs_median_low_pass', y='Z_abs_median_high_pass', 
    data = combined_ventilated_1min_all, kind='scatter', color='blue', alpha = 0.1, 
    height=6, ratio=5, space=0.2, dropna=True, xlim=[0, 1.2], ylim=[0, 1.2])

plot.ax_joint.hlines(0.2, 0, 2, color = 'black', linestyle = 'dashed')
#plot.ax_joint.text(0.02, 1, 'r: %.2f\np<0.0001' % corr[0], fontsize = 14)
plot.ax_joint.grid('True')
plot.ax_joint.set_xlabel('vertical sustained acceleration (m/sec$^2$)', size = 14)
plot.ax_joint.set_ylabel('vertical vibration (m/sec$^2$)', size = 14)
plot.ax_joint.tick_params(axis='both', which='major', labelsize=14)

plt.savefig('%s/%s.%s' % (DIR_WRITE, 'Z_abs_acceleration_Z_abs_vibration_jointplot', filetype),
    dpi = dpi,  format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
dpi = 300
filetype = 'jpg'

a = combined_ventilated_1min_all.dropna(subset = ['length_median_low_pass','length_median_high_pass'])
corr = stats.spearmanr(a['length_median_low_pass'], a['length_median_high_pass'])

plot = sns.jointplot(x='length_median_low_pass', y='length_median_high_pass', 
    data = combined_ventilated_1min_all, kind='scatter', color='blue', alpha = 0.1, 
    height=6, ratio=5, space=0.2, dropna=True, xlim=[0, 2], ylim=[0, 2])

plot.ax_joint.hlines(0.3, 0, 2, color = 'black', linestyle = 'dashed')
plot.ax_joint.text(0.1, 1.7, 'r: %.2f\np<0.0001' % corr[0], fontsize = 14)
plot.ax_joint.grid('True')
plot.ax_joint.set_xlabel('Euclidean sustained acceleration (m/sec$^2$)', size = 14)
plot.ax_joint.set_ylabel('Euclidean vibration (m/sec$^2$)', size = 14)
plot.ax_joint.tick_params(axis='both', which='major', labelsize=14)

plt.savefig('%s/%s.%s' % (DIR_WRITE, 'length_acceleration_length_vibration_jointplot', filetype),
    dpi = dpi,  format = filetype, bbox_inches='tight', pad_inches=0.1);

### 15. Identify periods of vibration and sustained acceleration

- Periods of vibration: 1-min-median of the Euclidean length of the acceleration vector after passing through a high-pass filter is >=0.3m/sec2 (high vibration)

- Periods of 'sustained' acceleration: 1-min-median of the Euclidean length of the acceleration vector after passing through a high-pass filter is <0.3m/sec2 low vibration

In [ ]:
low_1_vibr = {}; high_1_vibr = {}

for patient in combined_ventilated_1min_all['patient'].unique():
    patient_frame = combined_ventilated_1min_all[combined_ventilated_1min_all['patient'] == patient]
    
    # Only consider minutes when there was at least 0.3 m/sec2 Euclidean vibration
    patient_frame = patient_frame[patient_frame['length_median_high_pass'] >=0.3]
    
    low_1_vibr[patient] = patient_frame.sort_values('length_median_high_pass', ascending = True)[:1]
    high_1_vibr[patient] = patient_frame.sort_values('length_median_high_pass', ascending = True)[-1:]

In [ ]:
low_1_accel = {}; high_1_accel = {}

for patient in combined_ventilated_1min_all['patient'].unique():
    patient_frame = combined_ventilated_1min_all[combined_ventilated_1min_all['patient'] == patient]
    
    # Only consider minutes when there was less than 0.3 m/sec2 Euclidean vibration
    patient_frame = patient_frame[patient_frame['length_median_high_pass'] <0.3]
    
    # Only include those recordings where there is at least two minutes to compare
    if len(patient_frame) >=2:
        low_1_accel[patient] = patient_frame.sort_values('length_median_low_pass', ascending = True)[:1]
        high_1_accel[patient] = patient_frame.sort_values('length_median_low_pass', ascending = True)[-1:]

### 16. Compare periods with vibration with periods of no acceleration 

#### A. All minutes (irrespective of ventilation modes and patient characteristics)

In [ ]:
def two_paired_test(arr_1, arr_2, par):
    return {     'parameter': par,
                 'low_mean':  round(arr_1[par].mean(), 2),
                 'low_median': round(arr_1[par].median(), 2),
                 'low_sd':    round(arr_1[par].std(), 2),
                 'low_min': round(arr_1[par].min(), 2),
                 'low_max': round(arr_1[par].max(), 2),
                 'high_mean': round(arr_2[par].mean(), 2),
                 'high_median': round(arr_2[par].median(), 2),
                 'high_sd':   round(arr_2[par].std(), 2),
                 'high_min': round(arr_2[par].min(), 2),
                 'high_max': round(arr_2[par].max(), 2),
                 'statistic': round(stats.ttest_rel(arr_1[par], arr_2[par])[0], 2),
                 'p-value':   stats.ttest_rel(arr_1[par], arr_2[par])[1],}

In [ ]:
low_1_vibr_frme = pd.concat(low_1_vibr).reset_index(level = 1)
low_1_vibr_frme.head();

In [ ]:
high_1_vibr_frme = pd.concat(high_1_vibr).reset_index(level = 1)
high_1_vibr_frme.head();

In [ ]:
columns_to_analyse_1 = ['length_median_high_pass',]

comparison_stats_1 = []
for column in columns_to_analyse_1:
    comparison_stats_1.append(two_paired_test(low_1_vibr_frme, high_1_vibr_frme, column))

comparison_stats_frme_1 = DataFrame(comparison_stats_1, index= columns_to_analyse_1)
comparison_stats_frme_1;

In [ ]:
columns_to_analyse_2 = [ 'FiO2_set_mean',   'Leak_median', 'MV_kg_mean', 'PIP_mean', 'VTemand_kg_mean', ]

comparison_stats_2 = []

for column in columns_to_analyse_2:
    comparison_stats_2.append(two_paired_test(low_1_vibr_frme, high_1_vibr_frme, column))

comparison_stats_frme_2 = DataFrame(comparison_stats_2, index= columns_to_analyse_2)
comparison_stats_frme_2;

In [ ]:
comparison_stats_frme = pd.concat([comparison_stats_frme_1, comparison_stats_frme_2])
comparison_stats_frme

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'vibr_stats_all_recs.xlsx'))
comparison_stats_frme.to_excel(writer, 'stats')
writer.save()

#### B. VG ventilation

During these minutes the set tidal volume `VG_set_kg_mean` has a specific value different from np.nan

In [ ]:
(len(combined_ventilated_1min_all), 
 sum(combined_ventilated_1min_all['VG_set_kg_mean'].notnull()),
 sum(combined_ventilated_1min_all['VG_set_kg_mean'].isnull()), 
 sum(combined_ventilated_1min_all['VG_set_kg_mean'] == 0))

In [ ]:
VG = combined_ventilated_1min_all[combined_ventilated_1min_all['VG_set_kg_mean'].notnull()].copy()
len(VG)

In [ ]:
len(VG['accel_rec'].unique()), len(VG['patient'].unique())

In [ ]:
low_1_VG = {}; high_1_VG = {}

for patient in VG['patient'].unique():
    patient_frame_VG = VG[VG['patient'] == patient]
    
    # Only consider minutes when there was at least 0.3 m/sec2 vibration
    patient_frame_VG = patient_frame_VG[patient_frame_VG['length_median_high_pass'] >=0.3]
    
    low_1_VG[patient] = patient_frame_VG.sort_values('length_median_high_pass', ascending = True)[:1]
    high_1_VG[patient] = patient_frame_VG.sort_values('length_median_high_pass', ascending = True)[-1:]

low_1_frme_VG = pd.concat(low_1_VG)
high_1_frme_VG = pd.concat(high_1_VG)

In [ ]:
columns_to_analyse_1_VG = ['length_median_high_pass',]

comparison_stats_1_VG = []

for column in columns_to_analyse_1_VG:
    comparison_stats_1_VG.append(two_paired_test(low_1_frme_VG, high_1_frme_VG, column))

comparison_stats_frme_1_VG = DataFrame(comparison_stats_1_VG, index = columns_to_analyse_1_VG)
comparison_stats_frme_1_VG;

In [ ]:
columns_to_analyse_2_VG = ['PIP_mean', 'VTemand_kg_mean', 'VTdiff_abs_median',  'Pdiff_VG_abs_median', ]

comparison_stats_2_VG = []
for column in columns_to_analyse_2_VG:
    comparison_stats_2_VG.append(two_paired_test(low_1_frme_VG, high_1_frme_VG, column))

comparison_stats_frme_2_VG = DataFrame(comparison_stats_2_VG, index = columns_to_analyse_2_VG)
comparison_stats_frme_2_VG;

In [ ]:
comparison_stats_frme_VG = pd.concat([comparison_stats_frme_1_VG, comparison_stats_frme_2_VG])
comparison_stats_frme_VG

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'vibr_stats_VG.xlsx'))
comparison_stats_frme_VG.to_excel(writer, 'stats')
writer.save()

#### C. Minutes with pressure limited (non-VG) ventilation

In [ ]:
Plim = combined_ventilated_1min_all[combined_ventilated_1min_all['VG_set_kg_mean'].isnull()].copy()
len(Plim)

In [ ]:
len(Plim['accel_rec'].unique()), len(Plim['patient'].unique())

In [ ]:
low_1_Plim= {}; high_1_Plim = {}

for patient in Plim['patient'].unique():
    patient_frame_Plim = Plim[Plim['patient'] == patient]
    
    # Only consider minutes when there was at least 0.3 m/sec2 vibration
    patient_frame_Plim = patient_frame_Plim[patient_frame_Plim['length_median_high_pass'] >=0.3]
    
    low_1_Plim[patient] = patient_frame_Plim.sort_values('length_median_high_pass', ascending = True)[:1]
    high_1_Plim[patient] = patient_frame_Plim.sort_values('length_median_high_pass', ascending = True)[-1:]

low_1_frme_Plim = pd.concat(low_1_Plim)
high_1_frme_Plim = pd.concat(high_1_Plim)

In [ ]:
columns_to_analyse_1_Plim = ['length_median_high_pass',]

comparison_stats_1_Plim = []

for column in columns_to_analyse_1_Plim:
    comparison_stats_1_Plim.append(two_paired_test(low_1_frme_Plim, high_1_frme_Plim, column))

comparison_stats_frme_1_Plim = DataFrame(comparison_stats_1_Plim, index = columns_to_analyse_1_Plim)
comparison_stats_frme_1_Plim;

In [ ]:
columns_to_analyse_2_Plim = ['PIP_mean', 'VTemand_kg_mean', 'Pdiff_noVG_abs_median',]

comparison_stats_2_Plim = []

for column in columns_to_analyse_2_Plim:
    comparison_stats_2_Plim.append(two_paired_test(low_1_frme_Plim, high_1_frme_Plim, column))

comparison_stats_frme_2_Plim = DataFrame(comparison_stats_2_Plim, index = columns_to_analyse_2_Plim)
comparison_stats_frme_2_Plim;

In [ ]:
comparison_stats_frme_Plim = pd.concat([comparison_stats_frme_1_Plim, comparison_stats_frme_2_Plim])
comparison_stats_frme_Plim

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'vibr_stats_Plim.xlsx'))
comparison_stats_frme_Plim.to_excel(writer, 'stats')
writer.save()

#### D. Minutes with SIPPV ventilation

In [ ]:
sippv = combined_ventilated_1min_all[combined_ventilated_1min_all['patient'].isin(sippv_only)].copy()
len(sippv)

In [ ]:
len(sippv['accel_rec'].unique()), len(sippv['patient'].unique())

In [ ]:
low_1_sippv= {}; high_1_sippv = {}

for patient in sippv['patient'].unique():
    patient_frame_sippv = sippv[sippv['patient'] == patient]
    
    # Only consider minutes when there was at least 0.3 m/sec2 vibration
    patient_frame_sippv = patient_frame_sippv[patient_frame_sippv['length_median_high_pass'] >=0.3]
   
    low_1_sippv[patient] = patient_frame_sippv.sort_values('length_median_high_pass', ascending = True)[:1]
    high_1_sippv[patient] = patient_frame_sippv.sort_values('length_median_high_pass', ascending = True)[-1:]

low_1_frme_sippv = pd.concat(low_1_sippv)
high_1_frme_sippv = pd.concat(high_1_sippv)

In [ ]:
columns_to_analyse_1_sippv = ['length_median_high_pass',]

comparison_stats_1_sippv = []

for column in columns_to_analyse_1_sippv:
    comparison_stats_1_sippv.append(two_paired_test(low_1_frme_sippv, high_1_frme_sippv, column))

comparison_stats_frme_1_sippv = DataFrame(comparison_stats_1_sippv, index = columns_to_analyse_1_sippv)
comparison_stats_frme_1_sippv;

In [ ]:
columns_to_analyse_2_sippv = ['RR_mean',  'RRdiff_abs_median', ]

comparison_stats_2_sippv = []

for column in columns_to_analyse_2_sippv:
    comparison_stats_2_sippv.append(two_paired_test(low_1_frme_sippv, high_1_frme_sippv, column))

comparison_stats_frme_2_sippv = DataFrame(comparison_stats_2_sippv, index = columns_to_analyse_2_sippv)
comparison_stats_frme_2_sippv;

In [ ]:
comparison_stats_frme_sippv = pd.concat([comparison_stats_frme_1_sippv, comparison_stats_frme_2_sippv])
comparison_stats_frme_sippv

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'vibr_stats_sippv.xlsx'))
comparison_stats_frme_sippv.to_excel(writer, 'stats')
writer.save()

#### E. Minutes with SIMV ventilation

In [ ]:
simv = combined_ventilated_1min_all[combined_ventilated_1min_all['patient'].isin(simv_only)].copy()
len(simv)

In [ ]:
len(simv['accel_rec'].unique()), len(simv['patient'].unique())

In [ ]:
low_1_simv= {}; high_1_simv = {}

for patient in simv['patient'].unique():
    patient_frame_simv = simv[simv['patient'] == patient]
    
    # Only consider minutes when there was at least 0.3 m/sec2 vibration
    patient_frame_simv = patient_frame_simv[patient_frame_simv['length_median_high_pass'] >=0.3]
    
    low_1_simv[patient] = patient_frame_simv.sort_values('length_median_high_pass', ascending = True)[:1]
    high_1_simv[patient] = patient_frame_simv.sort_values('length_median_high_pass', ascending = True)[-1:]

low_1_frme_simv = pd.concat(low_1_simv)
high_1_frme_simv = pd.concat(high_1_simv)

In [ ]:
columns_to_analyse_1_simv = ['length_median_high_pass', ]

comparison_stats_1_simv = []

for column in columns_to_analyse_1_simv:
    comparison_stats_1_simv.append(two_paired_test(low_1_frme_simv, high_1_frme_simv, column))

comparison_stats_frme_1_simv = DataFrame(comparison_stats_1_simv, index = columns_to_analyse_1_simv)
comparison_stats_frme_1_simv;

In [ ]:
columns_to_analyse_2_simv = ['VTemand_kg_mean', 'VTemand_resp_kg_mean', 'VTespon_pat_kg_mean', ]

comparison_stats_2_simv = []

for column in columns_to_analyse_2_simv:
    comparison_stats_2_simv.append(two_paired_test(low_1_frme_simv, high_1_frme_simv, column))

comparison_stats_frme_2_simv = DataFrame(comparison_stats_2_simv, index = columns_to_analyse_2_simv)
comparison_stats_frme_2_simv;

In [ ]:
comparison_stats_frme_simv = pd.concat([comparison_stats_frme_1_simv, comparison_stats_frme_2_simv])
comparison_stats_frme_simv

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'vibr_stats_simv.xlsx'))
comparison_stats_frme_simv.to_excel(writer, 'stats')
writer.save()

#### E. Effect of gestation

<29 weeks corrected gestational age, all ventilated data

In [ ]:
vent_data_below_29 = combined_ventilated_1min_all[combined_ventilated_1min_all['patient'].isin(below_29)].copy()
len(vent_data_below_29)

In [ ]:
### All received VG ventilation

(len(vent_data_below_29), 
 sum(vent_data_below_29['VG_set_kg_mean'].notnull()),
 sum(vent_data_below_29['VG_set_kg_mean'].isnull()), 
 sum(vent_data_below_29['VG_set_kg_mean'] == 0))

In [ ]:
vent_data_below_29['VTdiff'] = vent_data_below_29['VTemand_kg_mean'] - vent_data_below_29['VG_set_kg_mean']
vent_data_below_29['VTdiff_abs'] = np.abs(vent_data_below_29['VTemand_kg_mean'] - vent_data_below_29['VG_set_kg_mean'])
vent_data_below_29['Pdiff_VG'] = vent_data_below_29['PIP_set_mean'] - vent_data_below_29['PIP_mean']

In [ ]:
vent_data_below_29.shape

In [ ]:
len(vent_data_below_29['accel_rec'].unique()), len(vent_data_below_29['patient'].unique())

In [ ]:
low_1_vent_data_below_29= {}; high_1_vent_data_below_29 = {}

for patient in vent_data_below_29['patient'].unique():
    patient_frame_vent_data_below_29 = vent_data_below_29[vent_data_below_29['patient'] == patient]
    
    # Only consider minutes when there was at least 0.3 m/sec2 vibration
    patient_frame_vent_data_below_29 = \
        patient_frame_vent_data_below_29[patient_frame_vent_data_below_29['length_median_high_pass'] >=0.3]
    
    low_1_vent_data_below_29[patient] = patient_frame_vent_data_below_29.sort_values('length_median_high_pass', 
                                                                                ascending = True)[:1]
    high_1_vent_data_below_29[patient] = patient_frame_vent_data_below_29.sort_values('length_median_high_pass', 
                                                                                ascending = True)[-1:]

low_1_frme_vent_data_below_29 = pd.concat(low_1_vent_data_below_29)
high_1_frme_vent_data_below_29 = pd.concat(high_1_vent_data_below_29)

In [ ]:
columns_to_analyse_1_vent_data_below_29 = ['length_median_high_pass', ]

comparison_stats_1_vent_data_below_29 = []

for column in columns_to_analyse_1_vent_data_below_29:
    comparison_stats_1_vent_data_below_29.append(two_paired_test(low_1_frme_vent_data_below_29, 
                                                                 high_1_frme_vent_data_below_29, column))

comparison_stats_frme_1_vent_data_below_29 = DataFrame(comparison_stats_1_vent_data_below_29, 
                                                       index = columns_to_analyse_1_vent_data_below_29)
comparison_stats_frme_1_vent_data_below_29;

In [ ]:
columns_to_analyse_2_vent_data_below_29 = ['FiO2_set_mean',  'Leak_median', 
                                           'MV_kg_mean', 'PIP_mean',  'VTemand_kg_mean',]

comparison_stats_2_vent_data_below_29 = []

for column in columns_to_analyse_2_vent_data_below_29:
    comparison_stats_2_vent_data_below_29.append(two_paired_test(low_1_frme_vent_data_below_29, 
                                                                 high_1_frme_vent_data_below_29, column))

comparison_stats_frme_2_vent_data_below_29 = DataFrame(comparison_stats_2_vent_data_below_29, 
                                                       index = columns_to_analyse_2_vent_data_below_29)
comparison_stats_frme_2_vent_data_below_29;

In [ ]:
comparison_stats_frme_vent_data_below_29 = pd.concat([comparison_stats_frme_1_vent_data_below_29, 
                                                       comparison_stats_frme_2_vent_data_below_29])
comparison_stats_frme_vent_data_below_29

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'vibr_stats_below_29weeks_recs.xlsx'))
comparison_stats_frme_vent_data_below_29.to_excel(writer, 'stats')
writer.save()

### 17. Compare periods with low and high sustained acceleration 
Compare the 1-1 minute with the lowest and highest sustained acceleration in each recording considering those minutes when there was no vibration (1-minute median of Euclidean acceleration was less than 0.3 m/sec2 in the data passed through a high pass filter).
This includes periods with the ambulance was not moving and also periods which were dominated by true acceleration.

In [ ]:
len(low_1_accel), len(high_1_accel)

In [ ]:
low_1_accel_frame = pd.concat(low_1_accel).reset_index(level = 1)
low_1_accel_frame;

In [ ]:
high_1_accel_frame = pd.concat(high_1_accel).reset_index(level = 1)
high_1_accel_frame;

In [ ]:
len(low_1_accel_frame), len(high_1_accel_frame)

In [ ]:
(high_1_accel_frame['length_median_low_pass'] - low_1_accel_frame['length_median_low_pass']) 

In [ ]:
# Only include those recordings when the difference is at least 0.25 m.sec2
difference = (high_1_accel_frame['length_median_low_pass'] - low_1_accel_frame['length_median_low_pass'])
to_included_sustained_accel = difference[difference > 0.25].index
to_included_sustained_accel

In [ ]:
low_1_accel_frame = low_1_accel_frame.reindex(to_included_sustained_accel)
high_1_accel_frame = high_1_accel_frame.reindex(to_included_sustained_accel)
len(low_1_accel_frame), len(high_1_accel_frame)

In [ ]:
columns_to_analyse_1 = ['length_median_low_pass', ]

comparison_stats_1_accel = []
for column in columns_to_analyse_1:
    comparison_stats_1_accel.append(two_paired_test(low_1_accel_frame, high_1_accel_frame, column))

comparison_stats_frme_1_accel = DataFrame(comparison_stats_1_accel, index= columns_to_analyse_1)
comparison_stats_frme_1_accel;

In [ ]:
columns_to_analyse_2 = ['FiO2_set_median', 'Leak_median', 'MV_kg_mean','PIP_mean',  'VTemand_kg_mean',  ]

comparison_stats_2_accel = []
for column in columns_to_analyse_2:
    comparison_stats_2_accel.append(two_paired_test(low_1_accel_frame, high_1_accel_frame, column))

comparison_stats_frme_2_accel = DataFrame(comparison_stats_2_accel, index= columns_to_analyse_2)
comparison_stats_frme_2_accel;

In [ ]:
comparison_stats_frme_accel = pd.concat([comparison_stats_frme_1_accel, comparison_stats_frme_2_accel])
comparison_stats_frme_accel

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'sustained_acccel_stats_all_recs.xlsx'))
comparison_stats_frme_accel.to_excel(writer, 'stats')
writer.save()

### 18. Export the filtered DataFrames

In [ ]:
with open('%s/%s.pickle' % (DATA_DUMP, 'combined_ventilated_1min'), 'wb') as handle:
    pickle.dump(combined_ventilated_1min, handle, protocol=pickle.HIGHEST_PROTOCOL) 

with open('%s/%s.pickle' % (DATA_DUMP, 'low_1_vibr_frame'), 'wb') as handle:
    pickle.dump(low_1_vibr_frme, handle, protocol=pickle.HIGHEST_PROTOCOL) 

with open('%s/%s.pickle' % (DATA_DUMP, 'high_1_vibr_frame'), 'wb') as handle:
    pickle.dump(high_1_vibr_frme, handle, protocol=pickle.HIGHEST_PROTOCOL) 
                            
with open('%s/%s.pickle' % (DATA_DUMP, 'low_1_accel_frame'), 'wb') as handle:
    pickle.dump(low_1_accel_frame, handle, protocol=pickle.HIGHEST_PROTOCOL) 

with open('%s/%s.pickle' % (DATA_DUMP, 'high_1_accel_frame'), 'wb') as handle:
    pickle.dump(high_1_accel_frame, handle, protocol=pickle.HIGHEST_PROTOCOL)    

## Figures for the paper

### Figure 1

#### Figure 1 ABC

In [ ]:
rec = 'default__393.txt', 'AL000628'
dpi = 300 
filetype = 'jpg'
xlim = [-3, 3]
bins = 50
logyscale = False
color = 'blue'

fig, axes = plt.subplots(nrows=1, ncols=3, figsize = [12, 3], sharex=False, sharey=False)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=0.5)
    
ax_ravelled = axes.ravel()

for i, par in enumerate(['X', 'Y', 'Z']):
    ax = axes[i]
    data = accelero_ventilated[rec][par]
    # Remove outlier data outside the X axis scale
    data = data[(data > xlim[0]) & (data < xlim[1])]
    data.hist(ax = ax , bins=bins, color = color)
    ax.set_xlabel('m/sec$^2$', size = 10)
    ax.set_ylabel('count', size = 10)
    ax.set_title(par, size = 10)
    ax.set_xlim([*xlim])
    ax.set_ylim([0, 60000])
            
    if logyscale:
        ax.set_yscale('log')
    
fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1ABC', filetype), 
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)


#### Figure 1D

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['X', 'Y', 'Z']

medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, ax = plt.subplots(figsize = (4, 4))

plt.boxplot([accelero_stats_median['X_abs'], accelero_stats_median['Y_abs'], accelero_stats_median['Z_abs']], 
    widths = 0.3, whis = [5, 95], showfliers = False, showmeans = True, meanprops = meanpointprops, 
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, 
            flierprops = flierprops)

ax.hlines([1.1, 1.3], [1, 1], [2, 3], color = 'black')
ax.vlines([1, 2, 1, 3,], [1.1, 1.1, 1.3, 1.3], [1.05, 1.05, 1.25, 1.25], color = 'black')
ax.text(1.2, 1.15, 'p<0.001'); ax.text(1.55, 1.35, 'p<0.001')

ax.set_xticklabels(xticklabels)
ax.set_ylabel('Median acceleration of recordings \n (m/sec$^2$)', size = 12)
ax.set_ylim(0, 1.5)
ax.tick_params(axis='both', which='major', labelsize=12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1D', 'jpg'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,);

#### Figure 1E

In [ ]:
filetype = 'jpg'
dpi  = 300

fig = plt.figure(figsize = [6,6])
ax = fig.add_subplot(111, projection='3d')
ax.quiver(0, 0, 0, 3.5, 2.8, 3.5, linewidth = 2, color = 'black', linestyle = 'dashed', arrow_length_ratio = 0.1)
ax.quiver(0, 0, 0, 3.5, 0, 0, linewidth = 2, color = 'black', arrow_length_ratio = 0.1)
ax.quiver(0, 0, 0, 0, 2.8, 0, linewidth = 2, color = 'black', arrow_length_ratio = 0.1)
ax.quiver(0, 0, 0, 0, 0, 3.5, linewidth = 2, color = 'black', arrow_length_ratio = 0.1)

ax.quiver(0, 2.8, 0, 0, 0, 3.5, linewidth = 2, linestyle = 'dotted', color = 'black', arrow_length_ratio = 0.1,)
ax.quiver(0, 2.8, 3.5, 3.5, 0, 0, linewidth = 2, linestyle = 'dotted', color = 'black', arrow_length_ratio = 0.1,)

ax.set_xlim([0, 5]); ax.set_ylim([0,4]); ax.set_zlim([0, 4])
ax.set_xlabel('X (m/sec$^2$)'); ax.set_ylabel('Y (m/sec$^2$)'); ax.set_zlabel('Z (m/sec$^2$)')
ax.view_init(elev=20., azim = 200)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1E', filetype), 
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

#### Figure 1F

In [ ]:
rec = 'default__393.txt', 'AL000628'
bins = 50
xlim = [0, 4]
ylim = [0, 40000]
logyscale = False 
color = 'blue' 
dpi = 300 
filetype = 'jpg'

fig, ax = plt.subplots(figsize = [4, 4])
    
data = accelero_ventilated[rec]['length']
# Remove outlier data outside the X axis scale
data = data[(data > xlim[0]) & (data < xlim[1])]
data.hist(ax = ax, bins=bins, color = color)
ax.set_xlabel('m/sec$^2$', size = 12)
ax.set_ylabel('count', size = 12)
ax.set_title('Euclidean Length', size = 12)
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)
    
if logyscale:
    ax.set_yscale('log')
    
fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1F', filetype), 
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)
        

#### Figure 1 combined

In [ ]:
dpi = 300
filetype = 'pdf'

fig = plt.figure(figsize = [9,6], dpi = dpi)
fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, hspace=0.4, wspace=0.5)


#Fig 1A,B,C

rec = 'default__393.txt', 'AL000628'
xlim = [-3, 3]
bins = 50
logyscale = False
color = 'black'

for i, par in enumerate(['X', 'Y', 'Z']):
    ax = fig.add_subplot(2,3, i+1)
    data = accelero_ventilated[rec][par]
    # Remove outlier data outside the X axis scale
    data = data[(data > xlim[0]) & (data < xlim[1])]
    data.hist(ax = ax , bins=bins, color = color)
    ax.set_xlabel('m/sec$^2$', size = 10)
    ax.set_ylabel('count', size = 10)
    ax.set_title(par, size = 10)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.set_xlim([*xlim])
    ax.set_ylim([0, 60000])
    plt.grid(False)
    if logyscale:
        ax.set_yscale('log')

        
# Figure 1D

ax = fig.add_subplot(2,3,4,)

xticklabels = ['X', 'Y', 'Z']
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([accelero_stats_median['X_abs'], accelero_stats_median['Y_abs'], accelero_stats_median['Z_abs']], 
    widths = 0.3, whis = [5, 95], showfliers = False, showmeans = True, meanprops = meanpointprops, 
    medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, 
    flierprops = flierprops)

ax.hlines([1.1, 1.3], [1, 1], [2, 3], color = 'black')
ax.vlines([1, 2, 1, 3,], [1.1, 1.1, 1.3, 1.3], [1.05, 1.05, 1.25, 1.25], color = 'black')
ax.text(1.2, 1.15, 'p<0.001'); ax.text(1.55, 1.35, 'p<0.001')
ax.set_xticklabels(xticklabels)
ax.tick_params(axis='both', which='major', labelsize=10)
ax.set_ylabel('Median acceleration\n (m/sec$^2$)', size = 10)
ax.set_ylim(0, 1.5)
ax.tick_params(axis='both', which='major', labelsize=10)
plt.grid(False)


# Figure 1E

ax = fig.add_subplot(2,3,5, projection='3d')

ax.quiver(0, 0, 0, 3.5, 2.8, 3.5, linewidth = 1, color = 'black', linestyle = 'dashed', arrow_length_ratio = 0.15)
ax.quiver(0, 0, 0, 3.5, 0, 0, linewidth = 1, color = 'black', arrow_length_ratio = 0.15)
ax.quiver(0, 0, 0, 0, 2.8, 0, linewidth = 1, color = 'black', arrow_length_ratio = 0.15)
ax.quiver(0, 0, 0, 0, 0, 3.5, linewidth = 1, color = 'black', arrow_length_ratio = 0.15)

ax.quiver(0, 2.8, 0, 0, 0, 3.5, linewidth = 1, linestyle = 'dotted', color = 'black', arrow_length_ratio = 0.15,)
ax.quiver(0, 2.8, 3.5, 3.5, 0, 0, linewidth = 1, linestyle = 'dotted', color = 'black', arrow_length_ratio = 0.15,)

ax.set_xticks(range(0,6),)
ax.set_yticks(range(0,5),)
ax.set_zticks(range(0,5),)
ax.tick_params(axis='both', which='major', labelsize=10)
ax.set_xlim([0, 5]); ax.set_ylim([0,4]); ax.set_zlim([0, 4])
ax.set_xlabel('X (m/sec$^2$)', fontsize = 10)
ax.set_ylabel('Y (m/sec$^2$)', fontsize = 10)
ax.set_zlabel('Z (m/sec$^2$)', fontsize = 10)
ax.view_init(elev=20., azim = 200)


# Figure 1F

ax = fig.add_subplot(2,3,6)

rec = 'default__393.txt', 'AL000628'
bins = 50
xlim = [0, 4]
ylim = [0, 40000]
logyscale = False  
   
data = accelero_ventilated[rec]['length']
# Remove outlier data outside the X axis scale
data = data[(data > xlim[0]) & (data < xlim[1])]
data.hist(ax = ax, bins=bins, color = color)
ax.set_xlabel('m/sec$^2$', size = 10)
ax.set_ylabel('count', size = 10)
ax.tick_params(axis='both', which='major', labelsize=10)
ax.set_title('Euclidean Length', size = 10)
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)
plt.grid(False)
if logyscale:
    ax.set_yscale('log')
    
fig.text(0.03, 0.97, 'A', fontsize = 16)
fig.text(0.33, 0.97, 'B', fontsize = 16)
fig.text(0.62, 0.97, 'C', fontsize = 16)

fig.text(0.03, 0.47, 'D', fontsize = 16) 
fig.text(0.33, 0.47, 'E', fontsize = 16)
fig.text(0.62, 0.47, 'F', fontsize = 16)
    
fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1', filetype), 
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

### Figure 2

#### Figure 2A

In [ ]:
sns.set_style('whitegrid', {'axes.grid' : False})

In [ ]:
dpi = 300
filetype = 'jpg'

a = combined_ventilated_1min_all.dropna(subset = ['X_abs_median_low_pass','X_abs_median_high_pass'])
#corr = stats.spearmanr(a['X_abs_median_low_pass'], a['X_abs_median_high_pass'])

plot = sns.jointplot(x='X_abs_median_low_pass', y='X_abs_median_high_pass', 
    data = combined_ventilated_1min_all, kind='scatter', color='black', alpha = 0.1, 
    height=6, ratio=5, space=0.2, dropna=True, xlim=[0, 2], ylim=[0,0.8])

plot.ax_joint.hlines(0.15, 0, 2, color = 'black', linestyle = 'dashed')
#plot.ax_joint.text(0.1, 0.7, 'r: %.2f\np<0.0001' % corr[0], fontsize = 14)
plot.ax_joint.grid('True')
plot.ax_joint.set_xlabel('front-back sustained acceleration (m/sec$^2$)', size = 14)
plot.ax_joint.set_ylabel('front-back vibration (m/sec$^2$)', size = 14)
plot.ax_joint.tick_params(axis='both', which='major', labelsize=14)

plt.savefig('%s/%s.%s' % (DIR_WRITE, 'X_abs_acceleration_X_abs_vibration_jointplot', filetype),
    dpi = dpi,  format = filetype, bbox_inches='tight', pad_inches=0.1);

plt.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2A', filetype),
    dpi = dpi,  format = filetype, bbox_inches='tight', pad_inches=0.1);

#### Figure 2B

In [ ]:
dpi = 300
filetype = 'jpg'

a = combined_ventilated_1min_all.dropna(subset = ['Y_abs_median_low_pass','Y_abs_median_high_pass'])
#corr = stats.spearmanr(a['Y_abs_median_low_pass'], a['Y_abs_median_high_pass'])

plot = sns.jointplot(x='Y_abs_median_low_pass', y='Y_abs_median_high_pass', 
    data = combined_ventilated_1min_all, kind='scatter', color='black', alpha = 0.1, 
    height=6, ratio=5, space=0.2, dropna=True, xlim=[0, 1.2], ylim=[0,0.8])

plot.ax_joint.hlines(0.07, 0, 2, color = 'black', linestyle = 'dashed')
#plot.ax_joint.text(0.1, 0.7, 'r: %.2f\np<0.0001' % corr[0], fontsize = 14)
plot.ax_joint.grid('True')
plot.ax_joint.set_xlabel('side-to-side sustained acceleration (m/sec$^2$)', size = 14)
plot.ax_joint.set_ylabel('side-to-side vibration (m/sec$^2$)', size = 14)
plot.ax_joint.tick_params(axis='both', which='major', labelsize=14)

plt.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2B', filetype),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

#### Figure 2C

In [ ]:
dpi = 300
filetype = 'jpg'

a = combined_ventilated_1min_all.dropna(subset = ['Z_abs_median_low_pass','Z_abs_median_high_pass'])
#corr = stats.spearmanr(a['Z_abs_median_low_pass'], a['Z_abs_median_high_pass'])

plot = sns.jointplot(x='Z_abs_median_low_pass', y='Z_abs_median_high_pass', 
    data = combined_ventilated_1min_all, kind='scatter', color='black', alpha = 0.1, 
    height=6, ratio=5, space=0.2, dropna=True, xlim=[0, 1.2], ylim=[0, 1.2])

plot.ax_joint.hlines(0.2, 0, 2, color = 'black', linestyle = 'dashed')
#plot.ax_joint.text(0.02, 1, 'r: %.2f\np<0.0001' % corr[0], fontsize = 14)
plot.ax_joint.grid('True')
plot.ax_joint.set_xlabel('vertical sustained acceleration (m/sec$^2$)', size = 14)
plot.ax_joint.set_ylabel('vertical vibration (m/sec$^2$)', size = 14)
plot.ax_joint.tick_params(axis='both', which='major', labelsize=14)

plt.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2C', filetype),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

#### Figure 2D

In [ ]:
dpi = 300
filetype = 'jpg'

a = combined_ventilated_1min_all.dropna(subset = ['length_median_low_pass','length_median_high_pass'])
#corr = stats.spearmanr(a['length_median_low_pass'], a['length_median_high_pass'])

plot = sns.jointplot(x='length_median_low_pass', y='length_median_high_pass', 
    data = combined_ventilated_1min_all, kind='scatter', color='black', alpha = 0.1, 
    height=6, ratio=5, space=0.2, dropna=True, xlim=[0, 2], ylim=[0, 2])

plot.ax_joint.hlines(0.3, 0, 2, color = 'black', linestyle = 'dashed')
#plot.ax_joint.text(0.1, 1.7, 'r: %.2f\np<0.0001' % corr[0], fontsize = 14)
plot.ax_joint.grid('True')
plot.ax_joint.set_xlabel('Euclidean sustained acceleration (m/sec$^2$)', size = 14)
plot.ax_joint.set_ylabel('Euclidean vibration (m/sec$^2$)', size = 14)
plot.ax_joint.tick_params(axis='both', which='major', labelsize=14)

plt.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2D', filetype),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

### Supplementary Figure 1

In [ ]:
%%time

recs = sorted(accelero_ventilated.keys())
bins = 50 
xlim = [-3, 3] 
logyscale = False
color = 'blue'
dpi = 200 
filetype = 'pdf'

fig, axes = plt.subplots(nrows=len(recs) // 4+1, ncols=12, figsize = [60, 60], sharex=False, sharey=False)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.6, hspace=0.6)
    
ax_ravelled = axes.ravel()
i = 0
for rec in recs:
    for par in ['X', 'Y', 'Z']:
        ax = ax_ravelled[i]
        data = accelero_ventilated[rec][par]
        # Remove outlier data outside the X axis scale
        data = data[(data > xlim[0]) & (data < xlim[1])]
        data.hist(ax = ax , bins=bins, color = color)
        ax.set_xlabel('m/sec$^2$', size = 10)
        ax.set_ylabel('count', size = 10)
        ax.set_title('%s - %s' % (rec[1], par), size = 10)
        ax.set_xlim([*xlim])
        if logyscale:
            ax.set_yscale('log') 
        i+=1
    
fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_1', filetype), dpi = dpi, 
    format = filetype, bbox_inches='tight', pad_inches=0.1)

plt.close()

In [ ]:
%%time

recs = sorted(accelero_ventilated.keys())
bins = 50 
xlim = [-3, 3] 
logyscale = False
color = 'blue'
dpi = 200 
filetype = 'pdf'

fig, axes = plt.subplots(nrows=len(recs), ncols=3, figsize = [8, 3 * len(recs)], sharex=False, sharey=False)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.6, hspace=0.6)
    
ax_ravelled = axes.ravel()
i = 0
for rec in recs:
    for par in ['X', 'Y', 'Z']:
        ax = ax_ravelled[i]
        data = accelero_ventilated[rec][par]
        # Remove outlier data outside the X axis scale
        data = data[(data > xlim[0]) & (data < xlim[1])]
        data.hist(ax = ax , bins=bins, color = color)
        ax.set_xlabel('m/sec$^2$', size = 10)
        ax.set_ylabel('count', size = 10)
        ax.set_title('%s - %s' % (rec[1], par), size = 10)
        ax.set_xlim([*xlim])
        if logyscale:
            ax.set_yscale('log') 
        i+=1
    
fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_1_mod', filetype), dpi = dpi, 
    format = filetype, bbox_inches='tight', pad_inches=0.1)

plt.close()